<a href="https://colab.research.google.com/github/Vinit-source/piper1-gpl/blob/main/notebooks/Piper_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0) Colab runtime + GPU check

In [2]:
# Check GPU
import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

Python: 3.12.12
PyTorch: 2.9.0+cu126
CUDA available: True
Wed Dec 10 07:21:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |              

1) System packages (incl. eSpeak dev)

In [3]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
!pkg-config --modversion espeak-ng

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,844 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-se

2) Clone repo fresh

In [4]:
%cd /content
!rm -rf piper1-gpl
!git clone https://github.com/OHF-voice/piper1-gpl.git
%cd piper1-gpl
!git checkout -b 891b2bfa3725326a591cc78522933b31bdf437cf
!pwd

/content
Cloning into 'piper1-gpl'...
remote: Enumerating objects: 817, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 817 (delta 234), reused 207 (delta 207), pack-reused 526 (from 2)
Receiving objects: 100% (817/817), 4.69 MiB | 34.29 MiB/s, done.
Resolving deltas: 100% (460/460), done.
/content/piper1-gpl
Switched to a new branch '891b2bfa3725326a591cc78522933b31bdf437cf'
/content/piper1-gpl


6) (Optional) Mount Google Drive for datasets and outputs

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


3) Python deps (editable install, no venv in Colab)

In [7]:
!python3 -m pip install --upgrade pip setuptools wheel
!python3 -m pip install -e ".[train]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
Obtaining file:///content/piper1-gpl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/84

4) Build the Cython extension used for alignment

In [8]:
%cd /content/piper1-gpl
!chmod +x ./build_monotonic_align.sh
!./build_monotonic_align.sh

/content/piper1-gpl
Compiling /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
P

5) Dev build (repo mode)

In [9]:
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja

  Using cached scikit_build-0.18.1-py3-none-any.whl.metadata (18 kB)
  Using cached cmake-4.2.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.5 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached scikit_build-0.18.1-py3-none-any.whl (85 kB)
Using cached cmake-4.2.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (28.9 MB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Attempting uninstall: cmake
    Found existing installation: cmake 3.31.10
    Uninstalling cmake-3.31.10:
      Successfully uninstalled cmake-3.31.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [cmake]


In [10]:
%cd /content/piper1-gpl
!python3 setup.py build_ext --inplace -v

/content/piper1-gpl


--------------------------------------------------------------------------------
-- Trying 'Ninja' generator
--------------------------------
---------------------------
----------------------
-----------------
------------
-------
--
Not searching for unused variables given on the command line.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX

In [11]:
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.0/683.0 kB 13.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [onnxscript]


7) Set paths and training hyperparams

In [12]:
from pathlib import Path

# ==== CHANGE THESE ====
VOICE_NAME      = "my_colab_voice"
ESPEAK_VOICE    = "en-us"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 22050
BATCH_SIZE      = 8       # drop to 8 or 4 if you OOM

DATA_ROOT       = Path("/content/drive/MyDrive/Piper-POC-Training/")
AUDIO_DIR       = DATA_ROOT / "wavs"
CSV_PATH        = DATA_ROOT / "metadata.csv"

CACHE_DIR       = Path("/content/piper_cache")
CONFIG_PATH     = DATA_ROOT / f"{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = ""  # e.g., "/content/drive/MyDrive/piper_ckpts/en_US-lessac-medium.ckpt"

# Make sure dirs exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print("CSV exists:", CSV_PATH.exists())
print("Audio dir exists:", AUDIO_DIR.exists())
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

CSV exists: True
Audio dir exists: True
Cache dir: /content/piper_cache
Config will be written to: /content/drive/MyDrive/Piper-POC-Training/my_colab_voice.json


8) Quick sanity checks

In [11]:
!espeak-ng --voices | head -n 20

Pty Language       Age/Gender VoiceName          File                 Other Languages
 5  af              --/M      Afrikaans          gmw/af               
 5  am              --/M      Amharic            sem/am               
 5  an              --/M      Aragonese          roa/an               
 5  ar              --/M      Arabic             sem/ar               
 5  as              --/M      Assamese           inc/as               
 5  az              --/M      Azerbaijani        trk/az               
 5  ba              --/M      Bashkir            trk/ba               
 5  bg              --/M      Bulgarian          zls/bg               
 5  bn              --/M      Bengali            inc/bn               
 5  bpy             --/M      Bishnupriya_Manipuri inc/bpy              
 5  bs              --/M      Bosnian            zls/bs               
 5  ca              --/M      Catalan            roa/ca               
 5  cmn             --/M      Chinese_(Mandarin) sit/cmn    

In [12]:
import pandas as pd, io, os, textwrap

csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df)
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

    audio                                               text
0       1  Vegetables are exported to all the areas like ...
1       2  They are also used in autism therapy which hel...
2       3  Agriculturist Machaiah from Pulikotu village s...
3       4  Yo Yo Honey Singh cheering up a singer in Delh...
4       5  No F O Bs in Cotton Green railway station, pas...
5       6  The allegation that talks ended in a deadlock ...
6       7  Agril Extension D D G Doctor A K Singh emphasi...
7       8  Apart from their outlet in Kammanahalli, Samov...
8       9  Deepak Jasani, head of research at H D F C Sec...
9      10  He said that he will look into it, secretary o...
10     11  S C mulls appointing panels to look into Goa i...
11     12  Iran's state run media said the strike was car...
12     13  and wouldst thou have my heart o god of all li...
13     14  The centre imparts training and recruits ex se...
14     15  It was, moreover, in his opinion undoubtedly t...
15     16  They have gam

In [13]:
from huggingface_hub import hf_hub_download
import shutil
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def download_pretrained_checkpoint() -> str:
    """
    Download pretrained checkpoint from Hugging Face.

    Returns:
        str: Path to downloaded checkpoint

    Raises:
        RuntimeError: If download fails
    """
    config = {
        "base_dir": "/content",
        "hf_checkpoint_repo": "rhasspy/piper-checkpoints",
        "hf_checkpoint_path": "en/en_US/lessac/medium/epoch=2164-step=1355540.ckpt",
        "hf_config_path": "en/en_US/lessac/medium/config.json",
    }
    print(config)

    print("Downloading model and config...")
    local_model_path = f"{config['base_dir']}/checkpoints/{config['hf_checkpoint_path']}"
    local_config_path =  f"{config['base_dir']}/checkpoints/{config['hf_config_path']}"
    pretrained_ckpt_path = f"{config['base_dir']}/pretrained.ckpt"

    try:
      if not os.path.exists(local_model_path):
        logger.info("Downloading pretrained checkpoint from Hugging Face...")
        downloaded_path = hf_hub_download(
            repo_id=config['hf_checkpoint_repo'],
            filename=config['hf_checkpoint_path'],
            repo_type="dataset",
            local_dir=os.path.join(config['base_dir'], "checkpoints"),
        )
        print("Downloaded model to", downloaded_path)
        # Copy to expected location
        shutil.copy(downloaded_path, pretrained_ckpt_path)
        logger.info(f"Pretrained checkpoint downloaded to: {pretrained_ckpt_path}")

      # Also download the config for reference
      if not os.path.exists(local_config_path):
          hf_hub_download(
              repo_id=config['hf_checkpoint_repo'],
              filename=config['hf_config_path'],
              repo_type="dataset",
              local_dir=os.path.join(config['base_dir'], "checkpoints"),
          )
          print("Downloaded config")

      return pretrained_ckpt_path

    except Exception as e:
        logger.error(f"Failed to download pretrained checkpoint: {e}")
        raise RuntimeError(f"Failed to download pretrained checkpoint: {e}")

download_pretrained_checkpoint()

{'base_dir': '/content', 'hf_checkpoint_repo': 'rhasspy/piper-checkpoints', 'hf_checkpoint_path': 'en/en_US/lessac/medium/epoch=2164-step=1355540.ckpt', 'hf_config_path': 'en/en_US/lessac/medium/config.json'}


en/en_US/lessac/medium/epoch=2164-step=1(…):   0%|          | 0.00/846M [00:00<?, ?B/s]

Downloaded model to /content/checkpoints/en/en_US/lessac/medium/epoch=2164-step=1355540.ckpt


config.json: 0.00B [00:00, ?B/s]

Downloaded config


'/content/pretrained.ckpt'

9) Kick off training

In [14]:
!python3 -m piper.train fit \
  --data.voice_name "my_colab_voice" \
  --data.csv_path "/content/drive/MyDrive/Piper-POC-Training/metadata.csv" \
  --data.audio_dir "/content/drive/MyDrive/Piper-POC-Training/wavs" \
  --model.sample_rate 22050 \
  --data.espeak_voice "en-us" \
  --data.cache_dir "/content/piper_cache" \
  --data.config_path "/content/drive/MyDrive/Piper-POC-Training/my_colab_voice.json" \
  --data.batch_size 8 \
  --ckpt_path "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)
/usr/local/lib/python3.12/dist-packages/lightning/fabric/utilities/seed.py:44: No seed found, seed set to 0
Seed set to 0
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
💡 Tip: For seamless cl

In [16]:
!cp /content/piper1-gpl/lightning_logs/version_0/checkpoints/epoch=2186-step=88.ckpt /content/drive/MyDrive/Piper-POC-Training/output/epoch=2186-lessac-medium-step=88.ckpt

10. Export to onnx

In [18]:
# !python3 -m piper.train.export_onnx \
#   --checkpoint "/content/piper1-gpl/lightning_logs/version_0/checkpoints/epoch=2186-step=88.ckpt" \
#   --output-file "/content/model.onnx"

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/piper1-gpl/src/piper/train/export_onnx.py:92: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1210 06:52:14.069000 8730 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 15 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverag

In [14]:
# =============================================================================
# ENVIRONMENT MODE SELECTION
# =============================================================================
# Set COLAB = True for Google Drive mode (Proof of Concept)
# Set COLAB = False for AWS S3 mode (Production)

COLAB: bool = True  # Toggle between Colab (Google Drive) and AWS (S3) mode
TEST_PRETRAINED: bool = False # Toggle between testing the downloaded pretrained model vs testing the newly saved trained model.

In [15]:
# =============================================================================
# CONFIGURATION DATACLASS
# =============================================================================
# Centralized configuration for all pipeline parameters.
# All required fields must be set - no fallback values are used.

from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional
import os
import sys


@dataclass
class PiperConfig:
    """
    Configuration for Piper TTS fine-tuning pipeline.

    Attributes:
        colab_mode: Whether running in Colab (Google Drive) or AWS (S3) mode

    Google Drive Configuration (COLAB = True):
        gdrive_dataset_path: Path to dataset folder in Google Drive
        gdrive_output_path: Path to save outputs in Google Drive

    AWS S3 Configuration (COLAB = False):
        s3_bucket: S3 bucket name
        s3_dataset_prefix: S3 prefix for dataset
        s3_checkpoint_prefix: S3 prefix for checkpoints
        aws_region: AWS region
        aws_access_key_id: AWS access key (optional, uses IAM role if not set)
        aws_secret_access_key: AWS secret key (optional, uses IAM role if not set)

    Model Configuration:
        model_name: Name for the output model
        espeak_voice: eSpeak voice for phonemization
        sample_rate: Audio sample rate in Hz

    Training Configuration:
        batch_size: Training batch size
        max_epochs: Maximum training epochs
        validation_split: Fraction of data for validation
        num_test_examples: Number of test examples for audio generation
        learning_rate: Learning rate for fine-tuning
        precision: Training precision (e.g., "16-mixed", "32")
        checkpoint_epochs: Save checkpoint every N epochs
        device: Training device ("cpu" or "gpu")
        use_pretrained: Whether to use pretrained checkpoint
        resume_training: Whether to resume from existing checkpoint
    """

    # Environment mode
    colab_mode: bool = True

    # ==================== GOOGLE DRIVE CONFIGURATION ====================
    # Required when COLAB = True
    gdrive_dataset_path: str = "<GDRIVE_DATASET_PATH>"  # e.g., "/content/drive/MyDrive/Piper-POC-Training/"
    gdrive_output_path: str = "<GDRIVE_OUTPUT_PATH>"    # e.g., "/content/drive/MyDrive/Piper-POC-Training/output"

    # ==================== AWS S3 CONFIGURATION ====================
    # Required when COLAB = False
    s3_bucket: str = "<YOUR_S3_BUCKET_NAME>"           # e.g., "my-tts-training-bucket"
    s3_dataset_prefix: str = "<S3_DATASET_PATH>"       # e.g., "datasets/spicor"
    s3_checkpoint_prefix: str = "<S3_CHECKPOINT_PATH>" # e.g., "checkpoints/piper"
    aws_region: str = "<AWS_REGION>"                   # e.g., "us-east-1"
    aws_access_key_id: Optional[str] = None            # Leave None to use IAM role
    aws_secret_access_key: Optional[str] = None        # Leave None to use IAM role

    # ==================== MODEL CONFIGURATION ====================
    model_name: str = "<MODEL_NAME>"                   # e.g., "en_IN-spicor-medium"
    espeak_voice: str = "en-gb-x-rp"                        # eSpeak voice for phonemization
    sample_rate: int = 22050                           # Audio sample rate
    num_speakers: int = 1                              # Number of speakers (1 for single speaker)

    # ==================== TRAINING CONFIGURATION ====================
    batch_size: int = 8                                # Reduce if out of memory
    max_epochs: int = 4000                             # Maximum training epochs
    validation_split: float = 0.0                      # Validation split (0.0 to disable)
    num_test_examples: int = 0                         # Test examples for audio generation
    learning_rate: float = 1e-4                        # Learning rate
    precision: str = "32"                              # Training precision ("16-mixed" or "32")
    checkpoint_epochs: int = 200                       # Save checkpoint every N epochs
    device: str = "gpu"                                # "cpu" or "gpu"
    use_pretrained: bool = True                        # Use pretrained checkpoint
    resume_training: bool = False                      # Resume from existing checkpoint

    # ==================== LOCAL PATHS (Auto-configured) ====================
    base_dir: str = field(default="")
    local_dataset_dir: str = field(default="")
    local_wavs_dir: str = field(default="")
    local_cache_dir: str = field(default="")
    local_output_dir: str = field(default="")
    piper_dir: str = field(default="")

    # ==================== HUGGING FACE CHECKPOINT ====================
    hf_checkpoint_repo: str = "rhasspy/piper-checkpoints"
    hf_checkpoint_path: str = "en/en_US/ljspeech/high/ljspeech-2000.ckpt"
    hf_config_path: str = "en/en_US/ljspeech/high/config.json"

    def __post_init__(self):
        """Initialize derived paths based on environment mode."""
        if self.colab_mode:
            self.base_dir = "/content"
            self.piper_dir = "/content/piper1-gpl"
        else:
            self.base_dir = "./piper_training"
            self.piper_dir = "./piper1-gpl"

        self.local_dataset_dir = f"{self.base_dir}/dataset"
        self.local_wavs_dir = f"{self.local_dataset_dir}/wavs"
        self.local_cache_dir = f"{self.base_dir}/audio_cache"
        self.local_output_dir = f"{self.base_dir}/output/{self.model_name}"

    def validate(self) -> None:
        """
        Validate configuration and raise errors for missing required fields.
        No fallback values - all placeholders must be replaced.
        """
        errors = []

        # Validate model name
        if self.model_name == "<MODEL_NAME>" or not self.model_name:
            errors.append("model_name: Must be set (e.g., 'en_IN-spicor-medium')")

        if self.colab_mode:
            # Validate Google Drive configuration
            if self.gdrive_dataset_path == "<GDRIVE_DATASET_PATH>" or not self.gdrive_dataset_path:
                errors.append("gdrive_dataset_path: Must be set for COLAB mode")
            if self.gdrive_output_path == "<GDRIVE_OUTPUT_PATH>" or not self.gdrive_output_path:
                errors.append("gdrive_output_path: Must be set for COLAB mode")
        else:
            # Validate AWS S3 configuration
            if self.s3_bucket == "<YOUR_S3_BUCKET_NAME>" or not self.s3_bucket:
                errors.append("s3_bucket: Must be set for AWS mode")
            if self.s3_dataset_prefix == "<S3_DATASET_PATH>" or not self.s3_dataset_prefix:
                errors.append("s3_dataset_prefix: Must be set for AWS mode")
            if self.s3_checkpoint_prefix == "<S3_CHECKPOINT_PATH>" or not self.s3_checkpoint_prefix:
                errors.append("s3_checkpoint_prefix: Must be set for AWS mode")
            if self.aws_region == "<AWS_REGION>" or not self.aws_region:
                errors.append("aws_region: Must be set for AWS mode")

        # Validate training parameters
        if self.batch_size <= 0:
            errors.append("batch_size: Must be positive")
        if self.max_epochs <= 0:
            errors.append("max_epochs: Must be positive")
        if self.sample_rate <= 0:
            errors.append("sample_rate: Must be positive")
        if self.device not in ("cpu", "gpu"):
            errors.append("device: Must be 'cpu' or 'gpu'")

        if errors:
            error_msg = "Configuration validation failed:\n" + "\n".join(f"  - {e}" for e in errors)
            raise ValueError(error_msg)

    def create_directories(self) -> None:
        """Create all required local directories."""
        dirs = [
            self.local_dataset_dir,
            self.local_wavs_dir,
            self.local_cache_dir,
            self.local_output_dir,
        ]
        for d in dirs:
            Path(d).mkdir(parents=True, exist_ok=True)
            print(f"Created directory: {d}")

        # Create Google Drive output directory if in COLAB mode
        if self.colab_mode:
            Path(self.gdrive_output_path).mkdir(parents=True, exist_ok=True)
            print(f"Created Google Drive output directory: {self.gdrive_output_path}")


def detect_runtime_environment() -> str:
    """
    Detect the current runtime environment.

    Returns:
        str: 'colab', 'sagemaker', or 'local'
    """
    if 'google.colab' in sys.modules:
        return 'colab'
    elif os.environ.get('SM_CURRENT_HOST'):
        return 'sagemaker'
    return 'local'


# Display detected environment
RUNTIME_ENV = detect_runtime_environment()
print(f"Detected runtime environment: {RUNTIME_ENV}")
print(f"Mode: {'Google Colab (Google Drive)' if COLAB else 'Production (AWS S3)'}")

Detected runtime environment: colab
Mode: Google Colab (Google Drive)


In [16]:
# =============================================================================
# INITIALIZE CONFIGURATION
# =============================================================================
# Update the configuration values below before running the pipeline.
# All placeholder values (e.g., "<MODEL_NAME>") must be replaced.

config = PiperConfig(
    colab_mode=COLAB,

    # ----- Google Drive Configuration (for COLAB = True) -----
    gdrive_dataset_path="/content/drive/MyDrive/Piper-POC-Training/",
    gdrive_output_path="/content/drive/MyDrive/Piper-POC-Training/output",

    # ----- AWS S3 Configuration (for COLAB = False) -----
    s3_bucket="<YOUR_S3_BUCKET_NAME>",
    s3_dataset_prefix="<S3_DATASET_PATH>",
    s3_checkpoint_prefix="<S3_CHECKPOINT_PATH>",
    aws_region="<AWS_REGION>",
    aws_access_key_id=None,  # Set to None to use IAM role
    aws_secret_access_key=None,

    # ----- Model Configuration -----
    model_name="en_IN-spicor-medium",
    espeak_voice="en-gb",
    sample_rate=22050,
    num_speakers=1,  # 1 for single speaker

    # ----- Training Configuration -----
    batch_size=8,
    max_epochs=4000,
    validation_split=0.0,
    num_test_examples=0,
    learning_rate=1e-4,
    precision="32",
    checkpoint_epochs=200,
    device="cpu",
    use_pretrained=True,
    resume_training=False,
)

# Validate configuration - raises error if any required field is missing
config.validate()

# Create local directories
config.create_directories()

# Display configuration summary
print("\n" + "=" * 60)
print("CONFIGURATION SUMMARY")
print("=" * 60)
print(f"Mode: {'COLAB (Google Drive)' if config.colab_mode else 'AWS (S3)'}")
print(f"Model name: {config.model_name}")
print(f"Sample rate: {config.sample_rate}")
print(f"Batch size: {config.batch_size}")
print(f"Max epochs: {config.max_epochs}")
print(f"Device: {config.device}")
print(f"Use pretrained: {config.use_pretrained}")
print(f"Local output: {config.local_output_dir}")
print("=" * 60)

Created directory: /content/dataset
Created directory: /content/dataset/wavs
Created directory: /content/audio_cache
Created directory: /content/output/en_IN-spicor-medium
Created Google Drive output directory: /content/drive/MyDrive/Piper-POC-Training/output

CONFIGURATION SUMMARY
Mode: COLAB (Google Drive)
Model name: en_IN-spicor-medium
Sample rate: 22050
Batch size: 8
Max epochs: 4000
Device: cpu
Use pretrained: True
Local output: /content/output/en_IN-spicor-medium


In [17]:
# =============================================================================
# ONNX Exporter Class
# =============================================================================

import shutil
import subprocess
import os

class ONNXExporter:
    """
    Exports a trained Piper model checkpoint to ONNX format for production inference.

    This class handles:
    - Finding the best or latest checkpoint
    - Running the export subprocess
    - Copying the voice configuration file

    Attributes:
        config: PiperConfig instance
        checkpoint_path: Path to the checkpoint to export
        output_dir: Directory for ONNX output
    """

    def __init__(self, config: PiperConfig, checkpoint_path: str = None, output_dir: str = None):
        """
        Initialize the ONNX exporter.

        Args:
            config: PiperConfig instance
            checkpoint_path: Optional specific checkpoint path. If None, finds latest.
            output_dir: Optional specific output directory. If None, uses config default.
        """
        self.config = config
        self.output_dir = output_dir or config.onnx_output_dir
        self.checkpoint_path = checkpoint_path or self._find_checkpoint()

    def _find_checkpoint(self) -> str:
        """
        Find the checkpoint to export.

        Priority:
        1. Explicit checkpoint_path if provided
        2. Best checkpoint (epoch=*-step=*.ckpt)
        3. Latest checkpoint (last.ckpt)

        Returns:
            Path to the checkpoint file

        Raises:
            FileNotFoundError: If no checkpoint found
        """
        import glob

        checkpoint_dir = self.config.training_output_dir

        # Look for best checkpoint first
        best_checkpoints = glob.glob(os.path.join(checkpoint_dir, "epoch=*-step=*.ckpt"))
        if best_checkpoints:
            # Sort by modification time, get most recent
            best_checkpoints.sort(key=os.path.getmtime, reverse=True)
            return best_checkpoints[0]

        # Fall back to last.ckpt
        last_ckpt = os.path.join(checkpoint_dir, "last.ckpt")
        if os.path.exists(last_ckpt):
            return last_ckpt

        raise FileNotFoundError(f"No checkpoint found in {checkpoint_dir}")

    def export(self) -> str:
        """
        Export the checkpoint to ONNX format.

        Uses subprocess to run the Piper export script.

        Returns:
            Path to the exported ONNX file

        Raises:
            subprocess.CalledProcessError: If export fails
        """
        os.makedirs(self.output_dir, exist_ok=True)

        print(f"📦 Exporting checkpoint to ONNX...")
        print(f"   Checkpoint: {self.checkpoint_path}")
        print(f"   Output dir: {self.output_dir}")

        # Run export command
        cmd = [
            "python", "-m", "piper.train.export_onnx",
            "--checkpoint", self.checkpoint_path,
            "--output-file", os.path.join(self.output_dir, f"{self.config.model_name}.onnx")
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"❌ Export failed:")
            print(result.stderr)
            raise subprocess.CalledProcessError(result.returncode, cmd, result.stdout, result.stderr)

        print(result.stdout)

        # Find the exported ONNX file
        onnx_files = [f for f in os.listdir(self.output_dir) if f.endswith('.onnx')]
        if onnx_files:
            onnx_path = os.path.join(self.output_dir, onnx_files[0])
            print(f"✅ ONNX model exported: {onnx_path}")
            return onnx_path

        raise FileNotFoundError(f"No ONNX file found in {self.output_dir}")

    def copy_config(self) -> str:
        """
        Copy the voice configuration JSON to the output directory.

        The config.json file is required alongside the ONNX model for inference.

        Returns:
            Path to the copied config file
        """
        # Source config is in the training output directory
        source_config = os.path.join(self.config.training_output_dir, "config.json")

        # If not there, try the dataset directory
        if not os.path.exists(source_config):
            source_config = os.path.join(self.config.dataset_dir, "config.json")

        if os.path.exists(source_config):
            dest_config = os.path.join(self.output_dir, f"{self.config.voice_name}.onnx.json")
            shutil.copy(source_config, dest_config)
            print(f"✅ Config copied: {dest_config}")
            return dest_config
        else:
            print("⚠️ Warning: config.json not found, ONNX model may not work correctly")
            return None

print("✅ ONNXExporter class defined")

✅ ONNXExporter class defined


In [18]:
# =============================================================================
# Export Model to ONNX
# =============================================================================
# Set TEST_PRETRAINED = True to export the downloaded pretrained model
# Set TEST_PRETRAINED = False to export the newly trained model
try:

    # Use pretrained checkpoint from Hugging Face
    export_checkpoint_path = os.path.join("/content", "pretrained.ckpt")
    # Use the directory of the checkpoint as output directory
    export_output_dir = os.path.dirname(export_checkpoint_path)
    print("🔵 Exporting model to ONNX")
    print(f"   Checkpoint: {export_checkpoint_path}")


    exporter = ONNXExporter(config, checkpoint_path=export_checkpoint_path, output_dir=export_output_dir)
    onnx_path = exporter.export()
    config_path = exporter.copy_config()

    print("\n" + "=" * 50)
    print("📦 ONNX Export Complete!")
    print("=" * 50)
    print(f"ONNX Model: {onnx_path}")
    if config_path:
        print(f"Config: {config_path}")
    print("\nThese files can be used with Piper for inference.")

except FileNotFoundError as e:
    print(f"⚠️ Export skipped: {e}")
    if TEST_PRETRAINED:
        print("Make sure to download the pretrained checkpoint first (Section 5).")
    else:
        print("Train the model first to generate checkpoints.")

🔵 Exporting model to ONNX
   Checkpoint: /content/pretrained.ckpt
📦 Exporting checkpoint to ONNX...
   Checkpoint: /content/pretrained.ckpt
   Output dir: /content
❌ Export failed:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/piper1-gpl/src/piper/train/export_onnx.py", line 112, in <module>
    main()
  File "/content/piper1-gpl/src/piper/train/export_onnx.py", line 48, in main
    model = VitsModel.load_from_checkpoint(checkpoint_path, map_location="cpu")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/model_helpers.py", line 130, in wrapper
    return self.method(cls_type, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/module.py", line 1781, in load_from_checkpoint
    loaded = 

CalledProcessError: Command '['python', '-m', 'piper.train.export_onnx', '--checkpoint', '/content/pretrained.ckpt', '--output-file', '/content/en_IN-spicor-medium.onnx']' returned non-zero exit status 1.

In [ ]:
!cp /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json /content/model.onnx.json